In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import tensorflow as tf

# **Obtención de bases de datos**

La bases de datos se obtuvieron de https://www.gob.mx/salud/documentos/datos-abiertos-bases-historicas-direccion-general-de-epidemiologia
A pesar de contar con registro histórico, los años 2024 y 2023 se encuentran separados por mes, por lo que se concatenaron para trabajar sobre un solo csv anual.
Se usará una técnica análoga para trabajar por olas y para la general.

# **Creación de bases anuales**

NOTA: No es necesario correr los siguientes bloques, las bases anuales ya fueron generadas.

In [ ]:
# BASE ANUAL 2024

# Cargar ambos archivos csv en dataframes
db24ene = pd.read_csv('RUTA')
db24feb = pd.read_csv('RUTA')

# Concatenar los dfs eliminando el encabezado
df_concatenado = pd.concat([db24ene, db24feb], ignore_index=True)

# Guardar en un nuevo archivo
df_concatenado.to_csv('COVID_2024.csv', index=False)

In [ ]:
# BASE ANUAL 2023

db23ene = pd.read_csv('RUTA')
db23feb = pd.read_csv('RUTA')
db23mar = pd.read_csv('RUTA')
db23abr = pd.read_csv('RUTA')
db23may = pd.read_csv('RUTA')
db23jun = pd.read_csv('RUTA')
db23jul = pd.read_csv('RUTA')
db23ago = pd.read_csv('RUTA')
db23sep = pd.read_csv('RUTA')
db23oct = pd.read_csv('RUTA')
db23nov = pd.read_csv('RUTA')
db23dic = pd.read_csv('RUTA')

df_concatenado2 = pd.concat([db23ene, db23feb, db23mar, db23abr, db23may, db23jun, db23jul, db23ago, db23sep, db23oct, db23nov, db23dic], ignore_index=True)

df_concatenado2.to_csv('COVID_2023.csv', index=False)

Las bases anuales 2020, 2021 y 2022 ya se encuentran disponibles, por lo que no es necesario crearlas.

# **Análisis de valores nulos y desbalance de clases**

Los datos faltantes están representados con un valor numérico según lo descrito por el diccionario. Para representarlos utilizando la matriz de ausencia, es necesario cambiar dichos valores en cada variable según aplique.

In [ ]:
def reemplazar_a_nulos(df, columnas, valor_a_reemplazar):

    # columnas: Lista de nombres de columna
    # valor_a_reemplazar: Lista de valores que se cambiarán a NaN

    for col in columnas:
        df[col].replace(valor_a_reemplazar, np.nan, inplace=True)

    return df

In [ ]:
# Listas de variables, guiado por el archivo 'Catálogo' del diccionario

columnas_99 = ['SEXO', 'SECTOR', 'TIPO_PACIENTE']
valores_99 = [99]

columnas_98_99 = ['INTUBADO', 'OTRO_CASO', 'UCI', 'INDIGENA']
valores_98_99 = [98, 99]

columnas_97_98_99 = [
    'NEUMONIA', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR',
    'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO'
]
valores_97_98_99 = [97, 98, 99]

columnas_entidad = ['ENTIDAD_RES']
valores_entidad = [36, 97, 98, 99]

columnas_municipio = ['MUNICIPIO_RES']
valores_municipio = [999]

In [ ]:
# Ejemplo de uso

nulos_2020 = reemplazar_a_nulos(db2020, columnas_99, valores_99)
nulos_2020 = reemplazar_a_nulos(nulos_2020, columnas_98_99, valores_98_99)
nulos_2020 = reemplazar_a_nulos(nulos_2020, columnas_97_98_99, valores_97_98_99)
nulos_2020 = reemplazar_a_nulos(nulos_2020, columnas_entidad, valores_entidad)
nulos_2020 = reemplazar_a_nulos(nulos_2020, columnas_municipio, valores_municipio)


**Nullity matrix**

In [ ]:
def comparar_datos_faltantes(df):

    msno.matrix(df, figsize=(15, 8), sparkline=False, fontsize=8, labels=True)
    plt.title('Nullity Matrix: BD anual', fontsize=14)
    plt.show()

**Desbalance de clases**

De forma similar, podemos representar con la misma matriz la variable principal que causa un desbalance de clases grave, la cual es **FECHA_DEF**  
Recordando que el valor 9999-99-99 indica que el paciente no falleció, mostremos cuántos registros existen con dicho valor.

In [ ]:
def reemplazar_fecha_a_nulos(df, columnas, valor_a_reemplazar):

    for col in columnas:
        df[col] = df[col].astype(str)
        df[col].replace(valor_a_reemplazar, np.nan, inplace=True)

    return df

In [ ]:
columnas_defuncion = ['FECHA_DEF']
valores_defuncion = ['9999-99-99']

In [ ]:
# Ejemplo de uso

nulo_def_2020 = reemplazar_fecha_a_nulos(db2020, columnas_defuncion, valores_defuncion)

In [ ]:
comparar_datos_faltantes(nulo_def_2020)

**Dendograma**

In [ ]:
def dendrograma(df):

    msno.dendrogram(df)
    plt.title('Dendrograma de datos faltantes', fontsize=14)
    plt.show()

# **Preprocesamiento**

**1. Eliminación de casos no positivos**

Eliminar registros de pacientes no-positivos, esto corresponde a un valor de 4 o superior (hasta 7) en la variable CLASIFICACION_FINAL.
Deseamos hacerlo para todas las bases anuales.

In [ ]:
def eliminar_no_positivos(df):

    # Debug previo
    total_registros = df.shape[0]
    print("Registros antes de filtrar por casos positivos: ", total_registros)

    df = df[df['CLASIFICACION_FINAL'] < 4]

    # Debug posterior
    total_registros_filtrados = df_positivo.shape[0]
    print("Registros luego de filtrar por casos positivos: ", total_registros_filtrados)

    return df

Creación de archivo nuevo para reducir el tamaño y trabajar posteriormente

In [ ]:
df.to_csv('COVID_2020_CONFIRMADOS.csv', index=False)

Análisis de reducción de registros

*Base anual 2020*  
Registros antes de filtrar por casos positivos:  3,868,396  
Registros luego de filtrar por casos positivos:  1,563,135 (40.40%)

*Base anual 2021*  
Registros antes de filtrar por casos positivos:  8,830,345  
Registros luego de filtrar por casos positivos:  2,526,649 (28.61%)

*Base anual 2022*  
Registros antes de filtrar por casos positivos:  6,451,944  
Registros luego de filtrar por casos positivos:  3,195,409 (49.52%)

*Base enero 2023*  
Registros antes de filtrar por casos positivos:  6,669,733  
Registros luego de filtrar por casos positivos:  3,283,180 (49.22%)

*Base febrero 2023*  
Registros antes de filtrar por casos positivos:  6,884,449  
Registros luego de filtrar por casos positivos:  3,365,985 (48.89%)

*Base marzo 2023*  
Registros antes de filtrar por casos positivos:  7,070,534  
Registros luego de filtrar por casos positivos:  3,441,901 (48.67%)

*Base abril 2023*  
Registros antes de filtrar por casos positivos:  7,205,138  
Registros luego de filtrar por casos positivos:  3,489,417 (48.42%)

*Base mayo 2023*  
Registros antes de filtrar por casos positivos:  7,345,960  
Registros luego de filtrar por casos positivos:  3,530,121 (48.05%)

*Base junio 2023*  
Registros antes de filtrar por casos positivos:  954,148  
Registros luego de filtrar por casos positivos:  348,924 (36.56%)

*Base julio 2023*  
Registros antes de filtrar por casos positivos:  996,403  
Registros luego de filtrar por casos positivos:  360,525 (36.18%)

*Base agosto 2023*  
Registros antes de filtrar por casos positivos:  1,070,214  
Registros luego de filtrar por casos positivos:  391,292 (36.56%)

*Base septiembre 2023*  
Registros antes de filtrar por casos positivos:  1,155,416  
Registros luego de filtrar por casos positivos:  423,632 (36.66%)

*Base octubre 2023*  
Registros antes de filtrar por casos positivos:  1,183,124  
Registros luego de filtrar por casos positivos:  427,377 (36.12%)

*Base noviembre 2023*  
Registros antes de filtrar por casos positivos:  1,199,527  
Registros luego de filtrar por casos positivos:  427,756 (35.66%)

*Base diciembre de 2023*  
Registros antes de filtrar por casos positivos:  1,216,831  
Registros luego de filtrar por casos positivos:  428,177 (35.18%)

*Base anual 2023*  
Registros antes de filtrar por casos positivos:  42,951,477  
Registros luego de filtrar por casos positivos:  19,918,287 (46.37%)

Base anual 2024 (enero-febrero)  
Registros antes de filtrar por casos positivos:  2,486,160  
Registros luego de filtrar por casos positivos:  859,334 (34.56%)

**2. Selección de variables y acciones individuales**

Seleccionar las variables de interés para este trabajo, eliminación de registros y cambios de valores según aplique. La siguiente información se encuentra en *diccionario_datos_covid19*

La variables seleccionadas son:

1.   **SECTOR**: Identifica el tipo de institución del Sistema Nacional de Salud que brindó la atención. Tipo: int ([1-13], 99)
Se desea eliminar registros con 99 ("no especificado")
2.   **SEXO**: Identifica al sexo del paciente. Tipo: int (1-mujer, 2-hombre, 99-no especificado)
Eliminar registros con 99
3.   **ENTIDAD_RES**: Identifica la entidad de residencia del paciente. Tipo: int ([01-32], 36, 97, 98, 99)
Eliminar registros con 36 ("República Mexicana"), 97 ("no aplica"), 98 ("se ignora") y 99 ("no especificado")
4.   **MUNICIPIO_RES**: Identifica la entidad de residencia del paciente. Tipo: int (clave por municipio)
Eliminar registros con 999 ("no especificado")
5.   **TIPO_PACIENTE**: Identifica el tipo de atención que recibió el paciente en la unidad. Ambulatorio (regresó a casa) u hospitalizado. Tipo: int (1-ambulatorio, 2-hospitalizado, 99-no especificado)
Eliminar registros con 99
6.   **FECHA_SINTOMAS**: Identiifica la fecha en que inició la sintomatología del paciente. Tipo: string con formato AAAA-MM-DD
Se verificó que no existieran registros nulos
7.   **FECHA_DEF**: Identifica la fecha en que el paciente falleció. Tipo: string con formato AAAA-MM-DD
Se identificó que 9999-99-99 funge como ("no aplica")
8.   **INTUBADO**: Identifica si el paciente requirió de intubación. Tipo: int (1-si, 2-no, 97-no aplica, 98-se ignora, 99-no especificado)
Se cambiará cada instancia de 97 por 2, eliminar registros con 98 y 99
9.   **NEUMONIA**: Identifica si al paciente se le diagnosticó con neumonía. Tipo: int (1-si, 2-no, 97-no aplica, 98-se ignora, 99-no especificado)
Eliminar registros con 97, 98 y 99.
10.   **EDAD**: Identifica la edad del paciente. Tipo: int
Se aceptan registros con valor 0
11.   **DIABETES**: Identifica si el paciente tiene un diagnóstico de diabetes. Tipo: int (1-si, 2-no, 97-no aplica, 98-se ignora, 99-no especificado)
Eliminar registros con 97, 98 y 99.
12.   **EPOC**: Identifica si el paciente tiene un diagnóstico de EPOC. Tipo: int (1-si, 2-no, 97-no aplica, 98-se ignora, 99-no especificado)
Eliminar registros con 97, 98 y 99.
13.   **ASMA**: Identifica si el paciente tiene un diagnóstico de asma. Tipo: int (1-si, 2-no, 97-no aplica, 98-se ignora, 99-no especificado)
Eliminar registros con 97, 98 y 99.
14.   **INMUSUPR**: Identifica si el paciente presenta inmunosupresión. Tipo: int (1-si, 2-no, 97-no aplica, 98-se ignora, 99-no especificado)
Eliminar registros con 97, 98 y 99.
15.   **HIPERTENSION**: Identifica si el paciente tiene un diagnóstico de hipertensión.
Eliminar registros con 97, 98 y 99.
16.   **CARDIOVASCULAR**: Identifica si el paciente tiene un diagnóstico de enfermedades cardiovasculares.
Eliminar registros con 97, 98 y 99.
17.   **OBESIDAD**: Identifica si el paciente tiene diagnóstico de obesidad.
Eliminar registros con 97, 98 y 99.
18.   **RENAL_CRONICA**: Identifica si el paciente tiene diagnóstico de insuficiencia renal crónica.
Eliminar registros con 97, 98 y 99.
19.   **TABAQUISMO**: Identifica si el paciente tiene hábito de tabaquismo.
Eliminar registros con 97, 98 y 99.
20.   **OTRO_CASO**: Identifica si el paciente tuvo contacto con algún otro caso diagnósticado con SARS CoV-2
Se cambiará cada instancia de 97 por 2, eliminar registros con 98 y 99.
21.   **CLASIFICACION_FINAL**: Identifica si el paciente es un caso de COVID-19 según el catálogo "CLASIFICACION_FINAL". Tipo: int ([1-3]-positivo, [4-7]-negativo)
Utilizada como primer filtro para casos positivos
22.   **UCI**: Identifica si el paciente requirió ingresar a una Unidad de Cuidados Intensivos. Tipo: int (1-si, 2-no, 97-no aplica, 98-se ignora, 99-no especificado)
Se cambiará cada instancia de 97 por 2, eliminar registros con 98 y 99.




In [ ]:
# Selección de variables a conservar

variables_elegidas = ['SECTOR', 'SEXO', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
                       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
                       'EDAD', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
                       'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD',
                       'RENAL_CRONICA', 'TABAQUISMO', 'OTRO_CASO', 'UCI']

In [ ]:
def reducir_variables(df, variables):

    # variables: lista de columnas seleccionadas

    df_reducido = df.loc[:, variables].copy()
    return df_reducido

Eliminación de registros con valores indeseables

In [ ]:
def limpieza_registros(df):

    print("Número de registros antes del preprocesamiento:", len(df))

    remover_99 = ['SEXO', 'SECTOR', 'TIPO_PACIENTE']
    for col in remover_99:
        df = df[df[col] != 99]

    remover_98_99 = ['INTUBADO', 'OTRO_CASO', 'UCI']
    for col in remover_98_99:
        df = df[~df[col].isin([98, 99])]

    remover_97_98_99 = ['NEUMONIA', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION',
                       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO']
    for col in remover_97_98_99:
        df = df[~df[col].isin([97, 98, 99])]

    df = df[~df['ENTIDAD_RES'].isin([36, 97, 98, 99])]

    df = df[df['MUNICIPIO_RES'] != 999]

    reemplazar_97 = ['INTUBADO', 'OTRO_CASO', 'UCI']
    for col in reemplazar_97:
        df[col] = df[col].replace(97, 2)

    print("Número de registros después del preprocesamiento:", len(df))

    return df

In [ ]:
# Ejemplo de uso

db_var_red = reducir_variables(db2020_positivos, variables_elegidas)
db_reducida = limpieza_registros(db_var_red)
db_reducida.to_csv('COVID_2020_CONFIRMADOS_REDUCIDA')

# **3. Ingeniería de variables**

**3.1. Rangos de edad**

Se toma como referencia los rangos de edad proporcionados por la información general de Covid-19 en México
https://datos.covid-19.conacyt.mx/

En vez de rangos de edad cada 5 años, se consideran 10.

In [ ]:
def agregar_variables_edad(df):

    # Definir los rangos de edad

    df['0_9'] = (df['EDAD'] >= 0) & (df['EDAD'] <= 9)
    df['10_19'] = (df['EDAD'] >= 10) & (df['EDAD'] <= 19)
    df['20_29'] = (df['EDAD'] >= 20) & (df['EDAD'] <= 49)
    df['30_39'] = (df['EDAD'] >= 30) & (df['EDAD'] <= 39)
    df['40_49'] = (df['EDAD'] >= 40) & (df['EDAD'] <= 49)
    df['50_59'] = (df['EDAD'] >= 50) & (df['EDAD'] <= 59)
    df['60_69'] = (df['EDAD'] >= 60) & (df['EDAD'] <= 69)
    df['70_MAS'] = (df['EDAD'] >= 70)

    # Convertir los booleanos a enteros

    df['0_9'] = df['0_9'].astype(int)
    df['10_19'] = df['10_19'].astype(int)
    df['20_29'] = df['20_29'].astype(int)
    df['30_39'] = df['30_39'].astype(int)
    df['40_49'] = df['40_49'].astype(int)
    df['50_59'] = df['50_59'].astype(int)
    df['60_69'] = df['60_69'].astype(int)
    df['70_MAS'] = df['70_MAS'].astype(int)

    return df

**3.2. Asignación de valores de IDH por municipio**

La información obtenida está actualizada al último valor publicado por el programa de las Naciones Unidad para el Desarrollo (PNUD) en México.

Recurso: https://docs.google.com/spreadsheets/d/1HLYIfCnhQQ1Tm3JJgmpEAJrDD_TqRYxc/edit?gid=1279528506#gid=1279528506  
Informe: https://mexico.un.org/sites/default/files/2023-02/INFORME_PNUD_2022_electronico-Portadas-compressed.pdf

Para lo cual fue necesario crear una variable llamada **CLAVE_RES** que unifica las variables **ENTIDAD_RES** y **MUNICIPIO_RES** en ese orden, para así asignar en una nueva variable **IDH_VALOR** tal y como lo provee esta institución.

In [ ]:
# Leer el archivo de Excel con CLAVE_MUNICIPIO e IDH_2020

idh_datos = pd.read_excel('Resultados_IDH.xlsx', names=['CLAVE_MUNICIPIO', 'IDH_2020'], skiprows=0)

In [ ]:
def generar_clave_res(df):

    # Convertir las columnas a string y rellenar con ceros a la izquierda si es necesario

    df['ENTIDAD_RES'] = df['ENTIDAD_RES'].astype(str).str.zfill(2)  # Asegura que 'ENTIDAD_RES' tenga 2 dígitos
    df['MUNICIPIO_RES'] = df['MUNICIPIO_RES'].astype(str).str.zfill(3)  # Asegura que 'MUNICIPIO_RES' tenga 3 dígitos

    # Concatenar las dos columnas para crear 'CLAVE_RES'

    df['CLAVE_RES'] = df['ENTIDAD_RES'] + df['MUNICIPIO_RES']

    return df

In [ ]:
def agregar_idh_valor(df, idh_datos):

    # Asegurarse de que las columnas sean del tipo string de largo 5

    df['CLAVE_RES'] = df['CLAVE_RES'].astype(str).str.zfill(5).str.strip()
    idh_datos['CLAVE_MUNICIPIO'] = idh_datos['CLAVE_MUNICIPIO'].astype(str).str.zfill(5).str.strip()

    # Verificar los tipos de datos antes del merge

    print("Tipo de dato de CLAVE_RES:", df['CLAVE_RES'].dtype)
    print("Tipo de dato de CLAVE_MUNICIPIO:", idh_datos['CLAVE_MUNICIPIO'].dtype)

    # Realizar el merge utilizando CLAVE_RES y CLAVE_MUNICIPIO

    df_con_idh = df.merge(idh_datos, left_on='CLAVE_RES', right_on='CLAVE_MUNICIPIO', how='left')

    # Renombrar la columna IDH_2020 a IDH_VALOR

    df_con_idh.rename(columns={'IDH_2020': 'IDH_VALOR'}, inplace=True)

    # Eliminar la columna CLAVE_MUNICIPIO del merge

    df_con_idh.drop(columns=['CLAVE_MUNICIPIO'], inplace=True)

    # Número de registros antes de eliminar

    registros_antes = len(df_con_idh)
    print(f"Número de registros antes de eliminar NaN en IDH_VALOR: {registros_antes}")

    # Revisar los registros que resultaron en NaN

    claves_nan = df_con_idh[df_con_idh['IDH_VALOR'].isna()]['CLAVE_RES']
    print("Claves RES sin IDH_VALOR asignado:")
    print(claves_nan)

    if not claves_nan.empty:
        # Verificar claves problemáticas en el archivo IDH
        claves_en_idh = idh_datos[idh_datos['CLAVE_MUNICIPIO'].isin(claves_nan)]
        print("Claves presentes en el archivo de IDH:")
        print(claves_en_idh)

    # Eliminar registros que tienen NaN en IDH_VALOR

    df_con_idh = df_con_idh.dropna(subset=['IDH_VALOR'])

    # Número de registros después de eliminar

    registros_despues = len(df_con_idh)
    print(f"Número de registros después de eliminar NaN en IDH_VALOR: {registros_despues}")

    # Verificar si hay algún NaN en la columna IDH_VALOR

    if df_con_idh['IDH_VALOR'].isna().sum() == 0:
        print("Todos los registros en el dataframe tienen valores en IDH_VALOR.")
    else:
        print("Algunos registros todavía tienen NaN en IDH_VALOR.")

    return df_con_idh

In [ ]:
def confirmar_no_nulos(df):

    registros_antes = len(df)
    print(f"Registros antes de eliminar NaN: {registros_antes}")

    df = df.dropna(subset=['IDH_VALOR'])

    registros_despues = len(df)
    print(f"Registros después de eliminar NaN: {registros_despues}")

    return df

Posteriormente, se crearon rangos para este valor, cuyas variables son:

*   **IDH_MUY_BAJO** Intervalo de [0, 0.7)
*   **IDH_BAJO** Intervalo de [0.7, 0.75)
*   **IDH_MEDIO** Intervalo de [0.75, 0.8)
*   **IDH_MED_ALTO** Intervalo de [0.8, 0.825)
*   **IDH_ALTO** Intervalo de [0.825, 0.85)
*   **IDH_MUY_ALTO** Intervalo de [0.85, 1]

In [ ]:
def asignar_rango_idh(df):

    # Convertir la columna a tipo numérico

    df['IDH_VALOR'] = pd.to_numeric(df['IDH_VALOR'], errors='coerce')

    # Definir los rangos de IDH

    df['IDH_MUY_BAJO'] = (df['IDH_VALOR'] >= 0) & (df['IDH_VALOR'] < 0.7)
    df['IDH_BAJO'] = (df['IDH_VALOR'] >= 0.7) & (df['IDH_VALOR'] < 0.75)
    df['IDH_MEDIO'] = (df['IDH_VALOR'] >= 0.75) & (df['IDH_VALOR'] < 0.8)
    df['IDH_MED_ALTO'] = (df['IDH_VALOR'] >= 0.8) & (df['IDH_VALOR'] < 0.825)
    df['IDH_ALTO'] = (df['IDH_VALOR'] >= 0.825) & (df['IDH_VALOR'] < 0.85)
    df['IDH_MUY_ALTO'] = (df['IDH_VALOR'] >= 0.85) & (df['IDH_VALOR'] <= 1)

    # Convertir los booleanos a enteros

    df['IDH_MUY_BAJO'] = df['IDH_MUY_BAJO'].astype(int)
    df['IDH_BAJO'] = df['IDH_BAJO'].astype(int)
    df['IDH_MEDIO'] = df['IDH_MEDIO'].astype(int)
    df['IDH_MED_ALTO'] = df['IDH_MED_ALTO'].astype(int)
    df['IDH_ALTO'] = df['IDH_ALTO'].astype(int)
    df['IDH_MUY_ALTO'] = df['IDH_MUY_ALTO'].astype(int)

    return df

**3.3. Creación de variables por sector de salud**

Se agruparon las unidades médicas de atención por demanda en las variables:

*   **UM_IMSS**
*   **UM_ISSTE**
*   **UM_MILITAR** Incluye a las UMs de SEMAR y SEDENA
*   **UM_PRIVADA** Intervalo de [0.8, 0.825)
*   **UM_SSA** Intervalo de [0.825, 0.85)
*   **UM_OTRAS** Incluye a las UMs con menor incidencia: PEMEX, IMSS Bienestar (no incluída en UM_IMSS por ser el antiguo Seguro Popular), Hospital Estatal, Hospital Municipal, Cruz Roja, Hospital Universitario y DIF



In [ ]:
def generar_variables_sector(df):

    df['UM_IMSS'] = (df['SECTOR'] == 4).astype(int)
    df['UM_ISSTE'] = (df['SECTOR'] == 6).astype(int)
    df['UM_MILITAR'] = ((df['SECTOR'] == 10) | (df['SECTOR'] == 11)).astype(int)
    df['UM_PRIVADA'] = (df['SECTOR'] == 9).astype(int)
    df['UM_SSA'] = (df['SECTOR'] == 12).astype(int)
    df['UM_OTRAS'] = ((df['SECTOR'] == 1) | (df['SECTOR'] == 2) |
                      (df['SECTOR'] == 3) | (df['SECTOR'] == 5) |
                      (df['SECTOR'] == 7) | (df['SECTOR'] == 8) |
                      (df['SECTOR'] == 13)).astype(int)

    return df

**3.4. Creación de la variable objetivo**

La variable que buscaremos predecir será llamada **DEFUNCION**, la cual toma un valor de 1 cuando FECHA_DEF tiene cualquier otro valor que no sea '9999-99-99', ya que esto corresponde a la fecha del deceso de un paciente

In [ ]:
def generar_variable_defuncion(df):

    df['DEFUNCION'] = (df['FECHA_DEF'] != '9999-99-99').astype(int)

    return df

**3.5. Eliminación de variables en desuso**

In [ ]:
def drop_final(df):

    columnas_a_eliminar = ['ENTIDAD_RES', 'MUNICIPIO_RES', 'EDAD', 'SECTOR', 'FECHA_DEF']

    # Eliminar las columnas
    df = df.drop(columns=columnas_a_eliminar, errors='ignore')

    return df

3.6. Binarización de variables

Con el fin de preparar las bases para la entrada a los modelos, las variables se binarizaron de la siguiente forma:

*  **SEXO**: 1 (mujer) 0 (no mujer)
*  **TIPO_PACIENTE**: 1 (no hospitalizado) 0 (hospitalizado)
*  **INTUBADO**: 1 (si) 0 (no)
*  **NEUMONIA**: 1 (si) 0 (no)
*  **DIABETES**: 1 (si) 0 (no)
*  **EPOC**: 1 (si) 0 (no)
*  **ASMA**: 1 (si) 0 (no)
*  **INMUSUPR**: 1 (si) 0 (no)
*  **HIPERTENSION**: 1 (si) 0 (no)
*  **CARDIOVASCULAR**: 1 (si) 0 (no)
*  **OBESIDAD**: 1 (si) 0 (no)
*  **RENAL_CRONICA**: 1 (si) 0 (no)
*  **TABAQUISMO**: 1 (si) 0 (no)
*  **OTRO_CASO**: 1 (si) 0 (no)
*  **UCI**: 1 (si) 0 (no)

In [ ]:
def binarizar_variables(df):

    var_a_binarizar = [
        'SEXO', 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'DIABETES',
        'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR',
        'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', 'OTRO_CASO', 'UCI'
    ]

    df[var_a_binarizar] = df[var_a_binarizar].replace(2, 0).astype(int)

    return df

Ejemplo de uso

In [ ]:
# 1. Rangos de edad
df_ingenieria_var = agregar_variables_edad (db2023_positivos_reducida)

# 2. Generar claves de residencia
df_con_claves = generar_clave_res (df_ingenieria_var)

# 3. IDH por entidad
idh_datos = pd.read_excel ('Resultados_IDH.xlsx', names=['CLAVE_MUNICIPIO', 'IDH_2020'], skiprows=0)
df_idh_val = agregar_idh_valor (df_con_claves, idh_datos)

# 4. Rangos de IDH
df_idh_rango = asignar_rango_idh (df_idh_val)

# 5. Variables de sector de atención
df_sector = generar_variables_sector (df_idh_rango)

# 6. Variable objetivo
df_defuncion = generar_variable_defuncion (df_sector)

# 7. Drop de siguientes variables en desuso
df_drop = drop_final (df_defuncion)

# 8. Binarización
df_binarizada = binarizar_variables (df_drop)

# 9. No-nulidad en IDH
df_idh_no_nulo = confirmar_no_nulos (df_binarizada)

# df_idh_no_nulo es el df resultante de la ingeniería de características

# **Delimitación por olas y etapas de vacunación**


La inclusión de pacientes en estos periodos temporales se hizo en referencia a la variable **FECHA_SINTOMAS**.

Se utilizó información oficial proporcionada por la Secretaría de Salud para la delimitación por olas según la fecha
https://epidemiologia.salud.gob.mx/gobmx/salud/documentos/covid19/Info-04-23-Int_COVID-19.pdf  
Archivo más reciente subido en febrero de 2024 a la página:
https://coronavirus.gob.mx/analisis-situacional-de-la-epidemia-en-mexico/

*  1ª ola: SE 8 a SE 39 de 2020 (16 de febrero a 26 de septiembre de 2020)
*  2ª ola: SE 40 de 2020 a SE 15 de 2021 (27 de septiembre de 2020 a 17 de abril de 2021)
*  3ª ola: SE 23 a SE 42 de 2021 (6 de junio a 23 de octubre de 2021)
*  4ª ola: SE 51 de 2021 a la SE 9 de 2022 (19 de diciembre de 2021 a 5 de marzo de 2022)
*  5ª ola COVID-19: SE 22 de 2022 a la SE 33 de 2022 (29 de mayo a 20 de agosto de 2022)
*  6ª ola COVID-19: SE 49 de 2022 a la SE 4 de 2023 (4 de diciembre de 2022 a 28 de enero de 2023)

(SE: Semana Epidemiológica)

No se encuentra información oficial sobre la existencia de olas posteriores.

**Por olas**

In [ ]:
# Fechas de inicio/fin de cada ola

ola_1_inicio = '2020-02-16'
ola_1_fin = '2020-09-26'

ola_2_inicio = '2020-09-27'
ola_2_fin = '2021-04-17'

ola_3_inicio = '2021-06-06'
ola_3_fin = '2021-10-23'

ola_4_inicio = '2021-12-19'
ola_4_fin = '2022-03-05'

ola_5_inicio = '2022-05-29'
ola_5_fin = '2022-08-20'

ola_6_inicio = '2022-12-04'
ola_6_fin = '2023-01-28'

# División por olas (algunas requieren unir dos bd anuales distintas)

ola_1 = db2020[(db2020['FECHA_SINTOMAS'] >= ola_1_inicio) & (db2020['FECHA_SINTOMAS'] <= ola_1_fin)]

ola_2_db_1 = db2020[(db2020['FECHA_SINTOMAS'] >= ola_2_inicio)]
ola_2_db_2 = db2021[(db2021['FECHA_SINTOMAS'] <= ola_2_fin)]
ola_2 = pd.concat([ola_2_db_1, ola_2_db_2])

ola_3 = db2021[(db2021['FECHA_SINTOMAS'] >= ola_3_inicio) & (db2021['FECHA_SINTOMAS'] <= ola_3_fin)]

ola_4_db_1 = db2021[(db2021['FECHA_SINTOMAS'] >= ola_4_inicio)]
ola_4_db_2 = db2022[(db2022['FECHA_SINTOMAS'] <= ola_4_fin)]
ola_4 = pd.concat([ola_4_db_1, ola_4_db_2])

ola_5 = db2022[(db2022['FECHA_SINTOMAS'] >= ola_5_inicio) & (db2022['FECHA_SINTOMAS'] <= ola_5_fin)]

ola_6_db_1 = db2022[(db2022['FECHA_SINTOMAS'] >= ola_6_inicio)]
ola_6_db_2 = db2023[(db2023['FECHA_SINTOMAS'] <= ola_6_fin)]
ola_6 = pd.concat([ola_4_db_1, ola_4_db_2])

**Por etapas de vacunación**

In [ ]:
# Fechas de inicio/fin de cada etapa

etapa_1_inicio = '2020-12-24'
etapa_1_fin = '2021-02-14'

etapa_2_inicio = '2021-02-15'
etapa_2_fin = '2021-05-15'

etapa_3_inicio = '2021-05-16'
etapa_3_fin = '2021-06-15'

etapa_4_inicio = '2021-06-16'
etapa_4_fin = '2021-07-15'

etapa_5_inicio = '2021-07-16'
etapa_5_fin = '2022-03-31'

# División por etapas (algunas requieren unir dos bd anuales distintas)

etapa_1_db_1 = db2020[(db2020['FECHA_SINTOMAS'] >= etapa_1_inicio)]
etapa_1_db_2 = db2021[(db2021['FECHA_SINTOMAS'] <= etapa_1_fin)]
etapa_1 = pd.concat([etapa_1_db_1, etapa_1_db_2])

etapa_2 = db2021[(db2021['FECHA_SINTOMAS'] >= etapa_2_inicio) & (db2021['FECHA_SINTOMAS'] <= etapa_2_fin)]

etapa_3 = db2021[(db2021['FECHA_SINTOMAS'] >= etapa_3_inicio) & (db2021['FECHA_SINTOMAS'] <= etapa_3_fin)]

etapa_4 = db2021[(db2021['FECHA_SINTOMAS'] >= etapa_4_inicio) & (db2021['FECHA_SINTOMAS'] <= etapa_4_fin)]

etapa_5_db_1 = db2021[(db2021['FECHA_SINTOMAS'] >= etapa_5_inicio)]
etapa_5_db_2 = db2022[(db2022['FECHA_SINTOMAS'] <= etapa_5_fin)]
etapa_5 = pd.concat([etapa_5_db_1, etapa_5_db_2])